### Fine Tune Dreambooth Stable Diffusion with LORA

#### Dowload the data and push to S3

In [ ]:
import sagemaker
from sagemaker.pytorch import PyTorch


In [ ]:
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

In [ ]:
train_data_url = sess.upload_data(
    path="data",
    key_prefix="dreambooth/sd/dogs",
)

In [ ]:
print(f"Training data s3 path {train_data_url}")

#### Starting the training job

In [ ]:
base_job_name="dreambooth-fine-tuning-lora"

In [ ]:
hyperparameters = {}

hyperparameters["pretrained_model_name_or_path"] = "CompVis/stable-diffusion-v1-4"
hyperparameters["instance_data_dir"] = "/opt/ml/input/data/train"
#hyperparameters["instance_prompt"] = "\'a photo of sks dog\'"
hyperparameters["resolution"] = 512
hyperparameters["train_batch_size"] = 1
hyperparameters["gradient_accumulation_steps"] = 1
hyperparameters["learning_rate"] = 5e-6
hyperparameters["max_train_steps"] = 400

In [ ]:
estimator = PyTorch(
    base_job_name=base_job_name,
    source_dir="./scripts",
    entry_point="accelerate_launcher.py",
    role=role,
    framework_version="1.12.0",
    py_version="py38",
    instance_count=1,
    instance_type="ml.g5.2xlarge",
    hyperparameters=hyperparameters,
    disable_profiler=True
)

In [ ]:
estimator.fit({"train":train_data_url})